In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
%matplotlib inline

# --- DATA LOADING ---
try:
    words = open('names.txt', 'r').read().splitlines()
except FileNotFoundError:
    words = ['apple', 'banana', 'cherry', 'date', 'elderberry'] # Dummy fallback

# --- VOCABULARY ---
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

# --- DATASET BUILDING ---
block_size = 3 # Context length: 3 characters to predict the next

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # Sliding window
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

# 80% train, 10% dev, 10% test
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [ ]:
# --- HYPERPARAMETERS ---
emb_dim = 10  # Dimension of character embedding
hidden_neurons = 200  # Neurons in the hidden layer
g = torch.Generator().manual_seed(2147483647)

# --- WEIGHTS & BIASES ---
C = torch.randn((vocab_size, emb_dim), generator=g)
W1 = torch.randn((block_size * emb_dim, hidden_neurons), generator=g)
b1 = torch.randn(hidden_neurons, generator=g)
W2 = torch.randn((hidden_neurons, vocab_size), generator=g)
b2 = torch.randn(vocab_size, generator=g)

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

print(f"Total parameters: {sum(p.nelement() for p in parameters)}")

In [ ]:
lossi = []

for i in range(200000):
    # 1. Minibatch (Size 32)
    ix = torch.randint(0, Xtr.shape[0], (32,))

    # 2. Forward pass
    emb = C[Xtr[ix]]  # (32, 3, 10)
    # Concatenate embeddings: (32, 30)
    h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])  # Efficient Softmax + NLL

    # 3. Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # 4. Update (Learning rate decay)
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    if i % 10000 == 0:
        print(f"Step {i}: Loss {loss.item():.4f}")
    lossi.append(loss.log10().item())

In [ ]:
# --- DEV SET EVALUATION ---
with torch.no_grad():
    emb = C[Xdev]
    h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
    logits = h @ W2 + b2
    dev_loss = F.cross_entropy(logits, Ydev)
    print(f"Dev Loss: {dev_loss.item():.4f}")

# --- SAMPLING ---
for _ in range(10):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print("".join(itos[i] for i in out))